Импорт библиотек и чтение файла

In [2]:
import matplotlib.pyplot as plt
import math
import numpy as np
import random
import pandas as pd
from scipy import stats

with open('Москва_2021.txt', 'r') as file:
    ages = list(map(int, file.readlines()))

Задание 1 (а). Проверить нулевую гипотезу о нормальном распределении случайной величины «возраст»

In [77]:
# Разбиение данных на 7 равных групп
num_bins = 7
bins = np.linspace(start=int(min(ages)), stop=int(max(ages)), num=num_bins + 1, dtype=int)
print(f'Интервалы 7-ми групп: {bins}')

# Вычисление эмпирических частот для каждой группы
empirical_freq, _ = np.histogram(ages, bins=bins)
print(f'Эмпирические частоты групп: {empirical_freq}')

# Вычисление теоретических частот для каждой группы

# Вычисление середины интервалов групп
avg_bins = []
for i in range(0, len(bins)-1):
  avg_bins.append((bins[i]+bins[i+1])/2)
print(f'Середины интервалов групп: {avg_bins}')

# Вычисление выборочной средней
theoretical_mu=0
for i in range(len(avg_bins)-1):
  theoretical_mu += avg_bins[i] * empirical_freq[i] / len(ages)
print(f'Выборочная средняя: {theoretical_mu}')

# Вычисление выборочного среднего квадратического отклонения
sum1 = 0
for x, n in zip(avg_bins, empirical_freq):
  sum1 += n * (x - theoretical_mu)**2
theoretical_sigma = math.sqrt(sum1 / len(ages))
print(f'Выборочное среднее квадратическое отклонение: {theoretical_sigma}')

# Нормирование границ интервалов
norm_bins = []
for i in range(len(bins)):
  norm_bins.append(round(float((bins[i] - theoretical_mu)/theoretical_sigma), 2))
norm_bins[0] = -math.inf
norm_bins[-1] = math.inf
print(f'Нормированные интервалы: {norm_bins}')

# Вычисление функций Лапласа для каждой границы интервалов
fi = []
for i in range(len(norm_bins)):
  if (norm_bins[i] == -math.inf):
    fi.append(-0.5)
  elif (norm_bins[i] == math.inf):
    fi.append(0.5)
  else:
     fi.append(round(float(stats.norm.cdf(norm_bins[i]) - 0.5), 2))
print(f'Значение функции Лапласа на интервалах: {fi}')

# Вычисление вероятностей Pi попадания X в интервалы и сразу же нахождение теоретических частот
theoretical_freq = []
for i in range(len(fi) - 1):
   theoretical_freq.append(len(ages) * (fi[i+1] - fi[i]))
print(f'Теоретические частоты: {theoretical_freq}')

# Вычисление хи-квадрат наблюдаемого
chi2_stat = 0
for i in range(num_bins):
  chi2_stat += ( ( empirical_freq[i] - theoretical_freq[i] )**2 ) / ( theoretical_freq[i] )

# Вычисление критического значения
alpha = 0.05
critical_value = stats.chi2.ppf(alpha, df=num_bins - 1)

# Вывод результатов
print(f"Критерий хи-квадрат наблюдаемый: {chi2_stat}")
print(f"Критическое точка: {critical_value} \n")

# Проверка гипотезы
if chi2_stat > critical_value:
    print("Отвергаем нулевую гипотезу: данные не распределены нормально.")
else:
    print("Не отвергаем нулевую гипотезу: данные распределены нормально.")

Интервалы 7-ми групп: [14 22 30 39 47 56 64 73]
Эмпирические частоты групп: [3867 8571 7674 6452 3583 1362  914]
Середины интервалов групп: [np.float64(18.0), np.float64(26.0), np.float64(34.5), np.float64(43.0), np.float64(51.5), np.float64(60.0), np.float64(68.5)]
Выборочная средняя: 33.953844493106736
Выборочное среднее квадратическое отклонение: 12.633192165520528
Нормированные интервалы: [-inf, -0.95, -0.31, 0.4, 1.03, 1.75, 2.38, inf]
Значение функции Лапласа на интервалах: [-0.5, -0.33, -0.12, 0.16, 0.35, 0.46, 0.49, 0.5]
Теоретические частоты: [5511.91, 6808.830000000001, 9078.44, 6160.369999999999, 3566.5300000000016, 972.689999999999, 324.2300000000003]
Критерий хи-квадрат наблюдаемый: 2406.6996660929717
Критическое точка: 1.6353828943279067 

Отвергаем нулевую гипотезу: данные не распределены нормально.


(Код из Лабораторной работы №2 - генерация выборок и выборки средних из задания №1)

In [88]:
# Подсчет объема выборок
y = 0.95
# Φ(t) = 0.95/2 = 0,4750. По функции Лапласа:
t = 1.96
N = 32423
d = 3
D = 144.9169680648149

n = math.ceil(t**2*D/d**2)

print(f'Объем выборок: {n}')

# Функция для вычисления выборочной средней
def calculate_average(selection):
    total = 0
    for age in selection:
        total += age
    return total / len(selection)

# Генерация выборок и расчет выборочной средней
averages = []
selections = []
for _ in range(36):
    selection = random.choices(ages, k=n)
    selections.append(selection)
    average = calculate_average(selection)
    averages.append(average)

# Результаты
print(f'Выборочные средние: {averages}')
print(f'Выборки: {selections}')

# Сортируем массив averages и берем левую, правую границы
sortedAverages = sorted(averages)
leftBorder = math.floor(min(averages))
rightBorder = math.ceil(max(averages))
print(f'Правая граница: {rightBorder}')
print(f'Левая граница: {leftBorder}')

# Создаем массив значений интервалов
intervals1 = np.arange(leftBorder, rightBorder + 1)
intervals = []
for i in range(len(intervals1)):
    intervals.append(int(intervals1[i]))
empirical_freq_avg, _ = np.histogram(sortedAverages, bins = intervals)
print(f'Интервалы средних:{intervals}')
print(f'Частота средних на интервалах: {empirical_freq_avg}')

Объем выборок: 62
Выборочные средние: [35.564516129032256, 36.016129032258064, 37.83870967741935, 33.935483870967744, 33.53225806451613, 34.79032258064516, 34.064516129032256, 37.016129032258064, 33.483870967741936, 36.66129032258065, 32.193548387096776, 36.596774193548384, 32.88709677419355, 37.12903225806452, 35.20967741935484, 35.58064516129032, 34.11290322580645, 32.983870967741936, 33.29032258064516, 35.806451612903224, 36.403225806451616, 36.03225806451613, 34.145161290322584, 33.016129032258064, 35.58064516129032, 36.67741935483871, 36.45161290322581, 33.45161290322581, 37.37096774193548, 33.24193548387097, 37.16129032258065, 34.87096774193548, 36.193548387096776, 37.20967741935484, 33.33870967741935, 35.725806451612904]
Выборки: [[21, 29, 46, 19, 29, 50, 28, 40, 43, 58, 20, 48, 40, 31, 46, 48, 22, 35, 47, 66, 39, 30, 19, 28, 44, 20, 30, 45, 30, 28, 37, 26, 23, 46, 44, 25, 30, 27, 36, 25, 47, 69, 19, 22, 46, 30, 29, 43, 40, 69, 48, 19, 34, 27, 23, 20, 28, 35, 42, 33, 36, 48], [4

Задание 1 (б). Проверить нулевую гипотезу о нормальном распределении случайной величины «средний возраст»

In [89]:
# Вычисление теоретических частот для каждой группы

# Вычисление середины интервалов групп
avg_intervals = []
for i in range(len(intervals)-1):
  avg_intervals.append(float((intervals[i]+intervals[i+1])/2))
print(f'Середины интервалов групп: {avg_intervals}')

# Вычисление выборочной средней
avg_theoretical_mu=0
for i in range(len(avg_intervals)-1):
  avg_theoretical_mu += avg_intervals[i] * empirical_freq_avg[i] / len(sortedAverages)
print(f'Выборочная средняя: {avg_theoretical_mu}')

# Вычисление выборочного среднего квадратического отклонения
sum1 = 0
for x, n in zip(avg_intervals, empirical_freq_avg):
  sum1 += n * (x - avg_theoretical_mu)**2
avg_theoretical_sigma = math.sqrt(sum1 / len(sortedAverages))
print(f'Выборочное среднее квадратическое отклонение: {avg_theoretical_sigma}')

# Нормирование границ интервалов
norm_intervals = []
for i in range(len(intervals)):
  norm_intervals.append(round(float((intervals[i] - avg_theoretical_mu)/avg_theoretical_sigma), 2))
norm_intervals[0] = -math.inf
norm_intervals[-1] = math.inf
print(f'Нормированные интервалы: {norm_intervals}')

# Вычисление функций Лапласа для каждой границы интервалов
avg_fi = []
for i in range(len(norm_intervals)):
  if (norm_intervals[i] == -math.inf):
    avg_fi.append(-0.5)
  elif (norm_intervals[i] == math.inf):
    avg_fi.append(0.5)
  else:
     avg_fi.append(round(float(stats.norm.cdf(norm_intervals[i]) - 0.5), 2))
print(f'Значение функции Лапласа на интервалах: {avg_fi}')

# Вычисление вероятностей Pi попадания X в интервалы и сразу же нахождение теоретических частот
avg_theoretical_freq = []
for i in range(len(avg_fi) - 1):
   avg_theoretical_freq.append(len(sortedAverages) * (avg_fi[i+1] - avg_fi[i]))
print(f'Теоретические частоты: {avg_theoretical_freq}')

# Вычисление хи-квадрат наблюдаемого
avg_chi2_stat = 0
for i in range(len(intervals)-1):
  avg_chi2_stat += ( ( empirical_freq_avg[i] - avg_theoretical_freq[i] )**2 ) / ( avg_theoretical_freq[i] )

# Вычисление критического значения
alpha = 0.05
avg_critical_value = stats.chi2.ppf(alpha, df=len(intervals) - 1 - 1)

# Вывод результатов
print(f"Критерий хи-квадрат наблюдаемый: {avg_chi2_stat}")
print(f"Критическое точка: {avg_critical_value} \n")

# Проверка гипотезы
if avg_chi2_stat > critical_value:
    print("Отвергаем нулевую гипотезу: данные не распределены нормально.")
else:
    print("Не отвергаем нулевую гипотезу: данные распределены нормально.")

Середины интервалов групп: [32.5, 33.5, 34.5, 35.5, 36.5, 37.5]
Выборочная средняя: 28.972222222222225
Выборочное среднее квадратическое отклонение: 6.4538365468021786
Нормированные интервалы: [-inf, 0.62, 0.78, 0.93, 1.09, 1.24, inf]
Значение функции Лапласа на интервалах: [-0.5, 0.23, 0.28, 0.32, 0.36, 0.39, 0.5]
Теоретические частоты: [26.28, 1.8000000000000007, 1.4399999999999993, 1.4399999999999993, 1.080000000000001, 3.9599999999999995]
Критерий хи-квадрат наблюдаемый: 110.60930077025962
Критическое точка: 1.1454762260617692 

Отвергаем нулевую гипотезу: данные не распределены нормально.


Задание №2 (а). По двум выборкам из ЛР №2 проверить нулевую гипотезу о равенстве дисперсий генеральных совокупностей при уровне значимости 0,05 при конкурирующей гипотезе 𝐻1: 𝐷1 > 𝐷2

In [ ]:
# Выборки 1 и 2 соответственно: selections[0] и selections[1]


Задание №2 (б). По двум выборкам из ЛР №2 проверить нулевую гипотезу о равенстве дисперсий генеральных совокупностей при уровне значимости 0,05 при конкурирующей гипотезе 𝐻1:𝐷1 ≠ 𝐷2.

In [ ]:
# Выборки 1 и 2 соответственно: selections[0] и selections[1]
